# Analyze healthcare data in Spark and Python

Description: This notebook shows you the basic steps when analyzing healthcare data in a notebook.You will learn how to load a data file to a notebook, access this data file in an Object Storage, and then load the content of the data file into a pandas DataFrame. Furthermore, you will learn how to explore and analyze the data in the Data Frame and how to vizualize your results by using the matplotlib library.

In just a few notebook cells, data was ingested, manipulated, visualized and yielded insights. Much more capability, including machine learning, could be leveraged with IBM Bluemix and Data Science Experience. This is just the tip of the iceberg!

## Get Data

In this notebook, you will explore and analyze healthcare data in csv format.

To get the data set, first upload demo2.txt file in object storage. 

Goto -> "Apache Spark Objectstore", create a container, then add file from local computer to object store. You can add the CSV file by dragging the file to the Data Source pane in the notebook Palette in the notebook. The data file is listed on the Data Source pane and is saved in the Object Storage instance associated with your Analytics for Apache Spark service. The progress bar below the file in the Data Source panel indicates the status of the load process. In the next step, you will learn how to access files in Object Storage. Below is the data structure of the CSV file.

#### Data Set: Patient, Provider and Payment
#### Total Records: 4,319

 1. personId: string 
 2. year: integer 
 3. zip: string 
 4. providerName
 5. providerType
 6. clinicalCondition
 7. procedureCode
 8. visits
 9. netPay


## Import packages to the notebook

Import the requests, base64, StringIO, pandas and re packages to use in the notebook. also import matplotlib and pyspark library for spark and visualization.

In [1]:
!pip install matplotlib==1.4.3

In [2]:
import requests as re ## for http to access auth url
import StringIO as json ##for parsing the configuration
import pandas as pd ## for pandas dataframe 
from ibmdbpy import IdaDataBase ## for db2 connection
from pyspark.sql.types import * ##for spark sql
from pixiedust.display import * ##for display of different graphs

Pixiedust database opened successfully
Pixiedust version 0.69


## Set Credentials to Access Data

Because the csv file is located in Object Storage, you need to define a helper function to access the data file that you loaded. Run the following cell to define the method get_file_content():

In [3]:
def setConfig(credentials):
    prefix = "fs.swift.service." + credentials['name']
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + ".auth.url", credentials['auth_url']+'/v3/auth/tokens')
    hconf.set(prefix + ".auth.endpoint.prefix", "endpoints")
    hconf.set(prefix + ".tenant", credentials['project_id'])
    hconf.set(prefix + ".username", credentials['user_id'])
    hconf.set(prefix + ".password", credentials['password'])
    hconf.setInt(prefix + ".http.port", 8080)
    hconf.set(prefix + ".region", credentials['region'])
    hconf.setBoolean(prefix + ".public", True)

##  Insert data source credentials

To add the credentials to access the csv file which is stored in Object Storage, click the next code cell and select the Insert to code function below the data file in the Data Source pane in the notebook palette.

The credentials for accessing the precipitation.csv file are added to the cell as a Python dictionary. With these credentials, you can use the helper function to load the data file into a pandas.DataFrame.

Note: When you select the Insert to code function, a code cell with a dictionary is created for you. Adjust the credentials in the Python dictionary to correspond with the credentials inserted by the Insert to code function and run the dictionary code cell. The access credentials to the Object Storage instance in the dictionary are provided for convenience for later usage.

In [4]:
# The code was removed by DSX for sharing.

In [5]:
## call setConfig(conf) function to access your data file 
credentials_1 ['name'] = 'demo2'
setConfig(credentials_1)

In [6]:
# The code was removed by DSX for sharing.

In [7]:
## call setConfig(conf) function to access your data file 
credentials_2 ['name'] = 'zipcode'
setConfig(credentials_2)

## Dash DB Connection Setup
Set dash DB Connection with username and credentionals

In [8]:
!pip install --user future
!pip install --user lazy
!pip install --user jaydebeapi
!pip uninstall --yes ibmdbpy
!pip install ibmdbpy --user --ignore-installed --no-deps
!wget -O $HOME/.local/lib/python2.7/site-packages/ibmdbpy/db2jcc4.jar https://ibm.box.com/shared/static/lmhzyeslp1rqns04ue8dnhz2x7fb6nkc.zip

Uninstalling ibmdbpy-0.1.4:
  Successfully uninstalled ibmdbpy-0.1.4
  Using cached ibmdbpy-0.1.4-py2.py3-none-any.whl
--2017-04-05 04:44:08--  https://ibm.box.com/shared/static/lmhzyeslp1rqns04ue8dnhz2x7fb6nkc.zip
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/shared/static/lmhzyeslp1rqns04ue8dnhz2x7fb6nkc.zip [following]
--2017-04-05 04:44:08--  https://ibm.ent.box.com/shared/static/lmhzyeslp1rqns04ue8dnhz2x7fb6nkc.zip
Resolving ibm.ent.box.com (ibm.ent.box.com)... 107.152.26.211
Connecting to ibm.ent.box.com (ibm.ent.box.com)|107.152.26.211|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcloud.com/d/1/R9pgvuyr29lOEaIuXZjLIP8sEIxAkq_pNe2k1KoarMzOR04CxsaCLNUikjQw1KDFVw5jbhbX8ikJWC7d5uIrnGKcGoun7FjQ0JWamwuJ1frFgNweHDLNQZQQ0PlMZ4jl66QxGYHWt83kqhp8qqBpUF0NriRnZ8dSGk

## Give Username and password of dashDB
Give username,password,host url to access DB

In [9]:
##Here add your host url,password,username e.g IdaDataBase('jdbc:db2://<host url>:50000/BLUDB:user=<username>;password=<password>')
##dashdb = IdaDataBase('jdbc:db2://awh-yp-small03.services.dal.bluemix.net:50000/BLUDB:user=dash107553;password=jeSjvYiXkcxG')
##java -jar /gpfs/fs01/user/s496-129c6a05b7f3a2-eb1cec95f18d/.local/lib/python2.7/site-packages/ibmdbpy/db2jcc4.jar
from ibmdbpy import IdaDataBase
dashdb = IdaDataBase('jdbc:db2://bluemix05.bluforcloud.com:50000/BLUDB:user=dash106639;password=oqebivwkshAU')


## Schema Initilization
We define a schema which we will use later when we load the data in.

In [10]:
customSchema = StructType([
    StructField("personId", StringType(), True),
    StructField("year",  IntegerType(), True),
    StructField("zip",  StringType(), True),
    StructField("providerName", StringType(), True),
    StructField("providerType", StringType(), True),
    StructField("clinicalCondition", StringType(), True),
    StructField("procedureCode", StringType(), True),
    StructField("visits",  IntegerType(), True),
    StructField("netPay", StringType(), True)])


zipCodeSchema = StructType([
    StructField("zip", IntegerType(), True),
    StructField("city",  StringType(), True),
    StructField("state",  StringType(), True),
    StructField("latitude", FloatType(), True),
    StructField("longitude", FloatType(), True)])

## Initialize SQLContext and Create Spark DataFrame
With SQLContext and a loaded local DataFrame, we create a Spark DataFrame from csv file:
To work with dataframes we need a SQLContext which is created using `SQLContext(sc)`. 
SQLContext uses SparkContext which has been already created, named `sc`. 

In [11]:
##spark sesstion is created to work with spark dataframe
sqlContext = SQLContext(sc)

In [12]:
df = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("delimiter", ";").schema(customSchema).load("swift://" + credentials_1['container'] + "." + credentials_1['name'] + "/" + credentials_1['filename'])

In [13]:
df1 = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("delimiter", ",").schema(zipCodeSchema).load("swift://" + credentials_2['container'] + "." + credentials_2['name'] + "/" + credentials_2['filename'])

In [14]:
##Run this commond to hide and clean all depriciation warnings
warnings.filterwarnings('ignore', category=DeprecationWarning, module='.*/IPython/.*')


## Run Queries in Spark dataframe


In [15]:
df.registerTempTable("ortho")
df1.registerTempTable("zipcodedata")
sqlContext.tableNames()

[u'zipcodedata', u'ortho']

In [ ]:
display(df)

personId,year,zip,providerName,providerType,clinicalCondition,procedureCode,visits,netPay
4218235523688,2014,97602,"Ben Alvey, D.C.",Chiropractor/DCM,"Spinal/Back Disord, Ex Low",97012 APPL MODALITY 1/> AREAS TRACTION MECHANICAL,24,$20.67
4218235523688,2014,97602,"Ben Alvey, D.C.",Chiropractor/DCM,"Spinal/Back Disord, Low Back",97012 APPL MODALITY 1/> AREAS TRACTION MECHANICAL,16,$18.25
4218235523688,2014,97602,"Ben Alvey, D.C.",Chiropractor/DCM,Fracture/Disloc - Hip/Fem Head,97012 APPL MODALITY 1/> AREAS TRACTION MECHANICAL,4,$19.00
4218235523688,2014,97602,"Ben Alvey, D.C.",Chiropractor/DCM,Fracture/Disloc - Hip/Fem Head,97010 APPLICATION MODALITY 1/> AREAS HOT/COLD PACKS,1,$28.00
4218235523688,2015,97602,"Ben Alvey, D.C.",Chiropractor/DCM,"Spinal/Back Disord, Ex Low",97012 APPL MODALITY 1/> AREAS TRACTION MECHANICAL,16,$22.75
4218235523688,2015,97602,"Ben Alvey, D.C.",Chiropractor/DCM,Fracture/Disloc - Hip/Fem Head,97012 APPL MODALITY 1/> AREAS TRACTION MECHANICAL,8,$22.00
4218235523688,2015,97602,"Ben Alvey, D.C.",Chiropractor/DCM,"Spinal/Back Disord, Low Back",97012 APPL MODALITY 1/> AREAS TRACTION MECHANICAL,7,$21.14
9478398715068,2014,~,"Milt Ragsdale, D.C.",Chiropractor/DCM,"Spinal/Back Disord, Low Back",97014 APPL MODALITY 1/> AREAS ELEC STIMJ UNATTENDED,31,$3.04
9478398715068,2014,~,Dr. Michael Barnes,Chiropractor/DCM,"Spinal/Back Disord, Low Back",97014 APPL MODALITY 1/> AREAS ELEC STIMJ UNATTENDED,8,$17.33
9478398715068,2015,~,Dr. Michael Barnes,Chiropractor/DCM,"Spinal/Back Disord, Low Back",97014 APPL MODALITY 1/> AREAS ELEC STIMJ UNATTENDED,35,$4.97


## Write Dataframe to dash DB 
Write spark sql dataframe into dashdb by converting these df into Pandas DF using ibmdbpy library of python.

In [17]:
pandas_df=df.toPandas() #Convert ot pandas dataframe

dashtbl= dashdb.as_idadataframe(pandas_df, "dashtbl3",clear_existing = True)##convert into ibmdbpyy dataframe 
#dashtbl= conn.as_idadataframe(pandas_df, "dashtbl3")##convert into ibmdbpyy dataframe 
dashdb.add_column_id(dashtbl, destructive = True )  ## write table into dashdb if not exist.
#conn.add_column_id(dashtbl, destructive = True)  ## write table into dashdb if not exist.

/gpfs/fs01/user/s98f-61d0e24b003d10-536547fd2c66/.local/lib/python2.7/site-packages/ibmdbpy/utils.py:238: UserWarning: Mixed case names are not supported in database object names.
  warnings.warn("Mixed case names are not supported in database object names.", UserWarning)


DataFrame will be splitted into 5 chunks. (888 rows per chunk)
Uploaded: 5/5... [DONE]


## Plot the results

When you work in notebooks, you can decide how to present your anlysis results and derived information. So far, you have used normal print functions, which are informative. However, you can also show your results in a visual way by using the popular matplotlib package to create plots.

## Visit by provider Types?

In [18]:
df10=sqlContext.sql("SELECT SUBSTR( providerType, 1,20) PROVIDER, to_date(concat(year,'-01-01')) year, sum(visits) VISITS FROM ortho group by providerType, year order by VISITS DESC LIMIT 5")
display(df10)

In [19]:
display(df10)

# Spending by Clinical Conditions

In [20]:
df20=sqlContext.sql("select substr(clinicalCondition,1,10) Condition, cast(sum(substr(netPay,2)) as INT) Amount from ortho group by clinicalCondition  order by clinicalCondition  ASC LIMIT 5")
display(df20)

# Paid Amount by Procedure Code

In [21]:
df30=sqlContext.sql(" select concat('ICD-',substr(procedureCode,1,5)) Code, cast(sum(substr(netPay,2)) as INT) Paid from ortho group by procedureCode  order by Paid desc LIMIT 5 ")
display(df30)

# Revenue by Year

In [22]:
df40=sqlContext.sql("select year, sum(substr(netPay,2)) Revenue from ortho group by year")
display(df40)

year,Revenue
2014,72628.62
2015,70260.28


## States Map Where the Providers Belongs

In [23]:
df50=sqlContext.sql("SELECT  o.providerName, o.zip, s.state, s.city,s.latitude,s.longitude,max(o.visits) Max_Visits FROM ortho o INNER JOIN zipcodedata s ON  o.zip = s.zip group by s.city,o.providerName,o.zip, s.state,s.latitude,s.longitude order by Max_Visits desc")
display(df50)

<IPython.core.display.Javascript object>


# Conclusion

### 1. Top visits are with three types of providers - Chiropractior/DCM, Rehabilitation Facil, and Therapists (Alternat
###  2. We spent the most on two clinical conditions are Bursitis and Arthopath
###  3. We can see paid amount by all procedure codes
###  4. Revenue in 2014 is USD 72,628 and the revenue in 2015 is USD 70,260 almost the same


### 5. We also plot the data into map with longtitude and latitude data of visit and see the visit in OR, MA and IN
### 6. We can also explore data in Cloudant and dashdb with the same data and create other mobile and web applications

# Thanks!